## Import

In [33]:
import pandas as pd
import numpy as np
import time
import psutil
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

## Metric Function

In [34]:
process = psutil.Process()

def metrics_start():
    return {"time": time.time(), "mem": process.memory_info().rss, "cpu": psutil.cpu_percent(interval=None)}

def metrics_end(start):
    end_time = time.time()
    end_mem = process.memory_info().rss
    end_cpu = psutil.cpu_percent(interval=None)
    return {"time": end_time - start["time"], "mem_used_MB": (end_mem - start["mem"]) / (1024**2), "cpu_percent": end_cpu}


## Dataset

In [38]:
df = pd.read_csv("./datasets/tesla_deliveries_dataset_2015_2025.csv")
print(df_pd.head())
print(df_pd.info())

   Year  Month         Region    Model  Estimated_Deliveries  \
0  2023      5         Europe  Model S                 17646   
1  2015      2           Asia  Model X                  3797   
2  2019      1  North America  Model X                  8411   
3  2021      2  North America  Model 3                  6555   
4  2016     12    Middle East  Model Y                 12374   

   Production_Units  Avg_Price_USD  Battery_Capacity_kWh  Range_km  \
0             17922       92874.27                   120       704   
1              4164       62205.65                    75       438   
2              9189      117887.32                    82       480   
3              7311       89294.91                   120       712   
4             13537      114846.78                   120       661   

   CO2_Saved_tons           Source_Type  Charging_Stations  
0         1863.42  Interpolated (Month)              12207  
1          249.46    Official (Quarter)               7640  
2          

## Analytics

In [39]:
metrics_pd = []

In [41]:
# ======================================
# Pandas 10 Analytics in One Cell
# ======================================

import pandas as pd
import numpy as np

analytics_results = []

# 1. Time-Series: Year-Month + 12M Rolling Avg
print("=== 1. Time-Series Analysis ===")
start = metrics_start()
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01')
ts = df.groupby('Date')['Estimated_Deliveries'].sum().to_frame('Monthly_Deliveries')
ts['Rolling_12M_Avg'] = ts['Monthly_Deliveries'].rolling(12).mean()
ts_metrics = metrics_end(start)
print(ts.head(3))
print("Metrics:", ts_metrics)
analytics_results.append(("Time-Series", ts_metrics))

# 2. Z-Score Anomaly Detection
print("=== 2. Z-Score Anomaly Detection ===")
start = metrics_start()
mean_del = df['Estimated_Deliveries'].mean()
std_del = df['Estimated_Deliveries'].std()
outliers = df[(df['Estimated_Deliveries'] - mean_del).abs()/std_del > 3]
outliers_metrics = metrics_end(start)
print(outliers.head(3))
print("Metrics:", outliers_metrics)
analytics_results.append(("Z-Score Anomaly", outliers_metrics))

# 3. Model-Level Z-Score
print("=== 3. Model Z-Score ===")
start = metrics_start()
df_model = df.copy()
df_model['mean_del'] = df_model.groupby('Model')['Estimated_Deliveries'].transform('mean')
df_model['std_del'] = df_model.groupby('Model')['Estimated_Deliveries'].transform('std')
df_model['Zscore'] = (df_model['Estimated_Deliveries'] - df_model['mean_del']) / df_model['std_del']
df_model_z = df_model[df_model['Zscore'].abs() > 3]
model_z_metrics = metrics_end(start)
print(df_model_z.head(3))
print("Metrics:", model_z_metrics)
analytics_results.append(("Model Z-Score", model_z_metrics))

# 4. Supply Chain Stress Index
print("=== 4. Supply Chain Stress Index ===")
start = metrics_start()
supply_chain = df.groupby('Year').agg({'Production_Units':'sum', 'Estimated_Deliveries':'sum'}).rename(
    columns={'Production_Units':'Prod', 'Estimated_Deliveries':'Del'}
)
supply_chain['Stress_Index'] = (supply_chain['Prod'] - supply_chain['Del']) / supply_chain['Del']
supply_chain_metrics = metrics_end(start)
print(supply_chain.head(3))
print("Metrics:", supply_chain_metrics)
analytics_results.append(("Supply Chain Stress", supply_chain_metrics))

# 5. CO2 Savings Intensity
print("=== 5. CO2 Savings Intensity ===")
start = metrics_start()
env_index = df.groupby('Model').agg(
    CO2_per_Car = ('CO2_Saved_tons','sum'),
    Total_Del = ('Estimated_Deliveries','sum'),
    Avg_Range = ('Range_km','mean')
)
env_index['CO2_per_Car'] = env_index['CO2_per_Car'] / env_index['Total_Del']
env_index = env_index.sort_values('CO2_per_Car', ascending=False)
env_index_metrics = metrics_end(start)
print(env_index.head(3))
print("Metrics:", env_index_metrics)
analytics_results.append(("CO2 Savings Intensity", env_index_metrics))

# 6. Linear Regression Forecast
print("=== 6. Linear Regression Forecast ===")
from sklearn.linear_model import LinearRegression

start = metrics_start()
yearly = df.groupby('Year')['Estimated_Deliveries'].sum().reset_index()
lr = LinearRegression()
lr.fit(yearly[['Year']], yearly['Estimated_Deliveries'])
future = pd.DataFrame({'Year':[2026,2027]})
future['Predicted_Deliveries'] = lr.predict(future[['Year']])
lr_metrics = metrics_end(start)
print(future)
print("Metrics:", lr_metrics)
analytics_results.append(("Linear Regression", lr_metrics))

# 7. Price Elasticity Correlation
print("=== 7. Price Elasticity Correlation ===")
start = metrics_start()
elasticity = df['Avg_Price_USD'].corr(df['Estimated_Deliveries'])
elasticity_metrics = metrics_end(start)
print("Price–Demand Correlation:", elasticity)
print("Metrics:", elasticity_metrics)
analytics_results.append(("Price Elasticity Correlation", elasticity_metrics))

# 8. K-Means Clustering
print("=== 8. K-Means Clustering ===")
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

start = metrics_start()
features = df[['Estimated_Deliveries','Battery_Capacity_kWh','Range_km','Avg_Price_USD']].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
kmeans = KMeans(n_clusters=4, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)
kmeans_metrics = metrics_end(start)
print(df[['Model','Cluster']].head(3))
print("Metrics:", kmeans_metrics)
analytics_results.append(("K-Means Clustering", kmeans_metrics))

# 9. Regional Performance Index
print("=== 9. Regional Performance Index ===")
start = metrics_start()
region_perf = df.groupby('Region').agg({'Estimated_Deliveries':'sum','CO2_Saved_tons':'sum','Range_km':'mean'})
region_perf['Score'] = region_perf['Estimated_Deliveries']*0.5 + region_perf['CO2_Saved_tons']*0.3 + region_perf['Range_km']*0.2
region_perf = region_perf.sort_values('Score', ascending=False)
region_metrics = metrics_end(start)
print(region_perf.head(3))
print("Metrics:", region_metrics)
analytics_results.append(("Regional Performance", region_metrics))

# 10. Model Lifecycle Analysis
print("=== 10. Model Lifecycle Analysis ===")
start = metrics_start()
lifecycle = df.sort_values(['Model','Year']).copy()
lifecycle['YoY_Del_Growth'] = lifecycle.groupby('Model')['Estimated_Deliveries'].pct_change()
lifecycle['YoY_Prod_Growth'] = lifecycle.groupby('Model')['Production_Units'].pct_change()
lifecycle_metrics = metrics_end(start)
print(lifecycle.head(3))
print("Metrics:", lifecycle_metrics)
analytics_results.append(("Model Lifecycle", lifecycle_metrics))


=== 1. Time-Series Analysis ===
            Monthly_Deliveries  Rolling_12M_Avg
Date                                           
2015-01-01              183180              NaN
2015-02-01              165053              NaN
2015-03-01              184567              NaN
Metrics: {'time': 0.00837087631225586, 'mem_used_MB': 0.0, 'cpu_percent': 9.1}
=== 2. Z-Score Anomaly Detection ===
      Year  Month         Region    Model  Estimated_Deliveries  \
281   2015      3           Asia  Model 3                 25410   
725   2015      6    Middle East  Model Y                 22315   
1341  2017     12  North America  Model 3                 25704   

      Production_Units  Avg_Price_USD  Battery_Capacity_kWh  Range_km  \
281              28802       73659.66                    60       341   
725              22673       81318.87                    82       472   
1341             28939       61247.57                   120       661   

      CO2_Saved_tons           Source_Type  Chargi

/opt/conda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


     Model  Cluster
0  Model S        3
1  Model X        0
2  Model X        2
Metrics: {'time': 0.3198230266571045, 'mem_used_MB': 0.0, 'cpu_percent': 96.8}
=== 9. Regional Performance Index ===
             Estimated_Deliveries  CO2_Saved_tons    Range_km         Score
Region                                                                     
Middle East               6698045       503797.14  502.543939  3.500262e+06
Asia                      6539935       484357.26  495.498485  3.415374e+06
Europe                    6494035       485135.79  499.662121  3.392658e+06
Metrics: {'time': 0.0026290416717529297, 'mem_used_MB': 0.0, 'cpu_percent': 100.0}
=== 10. Model Lifecycle Analysis ===
     Year  Month         Region       Model  Estimated_Deliveries  \
53   2015      3  North America  Cybertruck                  5716   
64   2015      7  North America  Cybertruck                  4405   
134  2015     10           Asia  Cybertruck                  5675   

     Production_Units  Avg

## Metrics

In [42]:
summary_df = pd.DataFrame([{"analytics": a[0], **a[1]} for a in analytics_results])
print("\n=== Pandas Analytics Summary ===")
print(summary_df)


=== Pandas Analytics Summary ===
                      analytics      time  mem_used_MB  cpu_percent
0                   Time-Series  0.008371        0.000          9.1
1               Z-Score Anomaly  0.001344        0.000         50.0
2                 Model Z-Score  0.006939        0.000         10.0
3           Supply Chain Stress  0.003026        0.000          0.0
4         CO2 Savings Intensity  0.003551        0.000          0.0
5             Linear Regression  0.003107        0.000         16.7
6  Price Elasticity Correlation  0.000426        0.000          0.0
7            K-Means Clustering  0.319823        0.000         96.8
8          Regional Performance  0.002629        0.000        100.0
9               Model Lifecycle  0.003837        0.125        100.0
